In [1]:
!pip install tabulate
from tabulate import tabulate

In [2]:
from run_experiments import plot_data, metrics, find_average
from matplotlib import pyplot as plt
from Models.MMC import MMC
from Models.HMC import HMC
from Models.DBN import FMC
from Models.model_sources.mtd_source import MTD
import pickle
import os
import itertools
from scipy import stats
methods = [HMC, FMC, MMC, MTD]

plt.rcParams.update({'font.size': 22})
metrics = ["Testing Accuracy", "Training Times"]

from os import walk

basepath = "/storage/data/"


method_perms = list(itertools.permutations(methods, 2))
method_perms = []
for m in methods:
    if m != MMC:
        method_perms.append((MMC, m))
headers = [f"{m1.__name__}-{m2.__name__}" for m1, m2 in method_perms]
for filename in os.listdir(basepath):
    path = basepath+filename
    
    metric_to_test = path.split("/")[-1].split("_")[1].split(".")[0]
    data_type = path.split("/")[-1].split("_")[0]
    
    with open(path, 'rb') as f:
        x = pickle.load(f)
    
    t_test = [headers, []]

    for st in x:
        for m1, m2 in method_perms:
            n1 = m1.__name__
            n2 = m2.__name__
            #print(f"T-Test for {n1} & {n2}")
            tstat, pval = stats.ttest_rel(x[st][n1]['Testing Accuracy'][0][0], x[st][n2]['Testing Accuracy'][0][0])
            
            if round(pval, 3) == 0:
                t_test[-1].append("0.000")
            else:
                t_test[-1].append(round(pval,3))
        t_test.append([])
    t_test.pop(-1)
    
    print(filename)
    #print(tabulate(t_test, headers="firstrow"))
    print(tabulate(t_test, headers="firstrow", tablefmt="latex"))
        
    for key, data in x.items():
        for method in data:
            for m in metrics:
                x[key][method][m] = [(find_average(x[key][method][m][0][0]), x[key][method][m][0][1])]

    methods = [HMC, FMC, MMC, MTD]
    types = [m.__name__ for m in methods]

    fig, axs = plt.subplots(len(metrics), 1, figsize=(20, 13))
    fig.suptitle(f'Data Type: {data_type}', y=1.08)
    colors = ["#21d185", "#d1218b", "#0000FF", "#FFA500"]
    for im, met in enumerate(metrics):
        plot_data(x.keys(), x, met, met, axs[im], colors, "state", types)
    
    axs[-1].set_xlabel(metric_to_test)
    fig.tight_layout()
    fig.savefig("experiment_results/"+data_type+"_"+metric_to_test+".png")
    #fig.show()

ModuleNotFoundError: No module named 'run_experiments'